In [40]:
import psycopg2
from dotenv import load_dotenv
import os
import pandas as pd
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns


# Loading the environment variables
load_dotenv('.env',override=True)

# =============================================================================
try:
    csv_file = r'Messy_HR_Dataset_Detailed.csv'
    df = pd.read_csv(csv_file)
except Exception as e:
    print(e)

In [41]:
print('*'*30,'\n')
print('Getting a sense of DF\n', df.head(), '\n')

****************************** 

Getting a sense of DF
    Unnamed: 0 FirstName LastName  StartDate ExitDate                    Title  \
0           0     Uriah  Bridges  20-Sep-19      NaN  Production Technician I   
1           1     Paula    Small  11-Feb-23      NaN  Production Technician I   
2           2    Edward     Buck  10-Dec-18      NaN       Area Sales Manager   
3           3   Michael  Riordan  21-Jun-21      NaN       Area Sales Manager   
4           4   Jasmine    Onque  29-Jun-19      NaN       Area Sales Manager   

        Supervisor                        ADEmail BusinessUnit EmployeeStatus  \
0     Peter Oneill    uriah.bridges@bilearner.com         CCDR         Active   
1  Renee Mccormick      paula.small@bilearner.com           EW         Active   
2   Crystal Walker      edward.buck@bilearner.com           PL         Active   
3   Rebekah Wright  michael.riordan@bilearner.com         CCDR         Active   
4        Jason Kim    jasmine.onque@bilearner.com   

In [42]:
print('*'*30,'\n')
print('Shape of DF: ', df.shape, '\n')

****************************** 

Shape of DF:  (3150, 39) 



In [43]:
print('*'*30,'\n')
print('DF Information: ', '\n')
df.info()
print('\n')

****************************** 

DF Information:  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3150 entries, 0 to 3149
Data columns (total 39 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  3150 non-null   int64  
 1   FirstName                   3150 non-null   object 
 2   LastName                    3150 non-null   object 
 3   StartDate                   3150 non-null   object 
 4   ExitDate                    1606 non-null   object 
 5   Title                       3150 non-null   object 
 6   Supervisor                  3150 non-null   object 
 7   ADEmail                     3150 non-null   object 
 8   BusinessUnit                3150 non-null   object 
 9   EmployeeStatus              3150 non-null   object 
 10  EmployeeType                3150 non-null   object 
 11  PayZone                     3150 non-null   object 
 12  EmployeeClassificationType  3150 non-n

In [44]:
print('*'*30,'\n')
print('DF Description: \n', df.describe(), '\n')

****************************** 

DF Description: 
         Unnamed: 0  LocationCode  Current Employee Rating  Employee ID  \
count  3150.000000   3150.000000              3150.000000  3150.000000   
mean   1495.748254  45155.332698                 2.968254  2502.748254   
std     867.007861  30019.418852                 1.020717   866.241991   
min       0.000000   1013.000000                 1.000000  1001.000000   
25%     746.000000  17616.500000                 2.000000  1750.250000   
50%    1495.500000  44455.500000                 3.000000  2504.500000   
75%    2246.750000  71713.250000                 3.000000  3254.750000   
max    2999.000000  98052.000000                 5.000000  4000.000000   

       Engagement Score  Satisfaction Score  Work-Life Balance Score  \
count       3150.000000         3150.000000              3150.000000   
mean           2.943492            3.021270                 2.992381   
std            1.430727            1.407413                 1.4092

In [45]:
col = df.columns.to_list()
print(col)
col_date = ['StartDate','ExitDate','DOB','Survey Date', 'Training Date']
for c in col_date:
    print('Column:', c)
    df[c] = pd.to_datetime(df[c],format='mixed')

print('*'*30,'\n')
print('DF Information: ', '\n')
df.info()
print('\n')

['Unnamed: 0', 'FirstName', 'LastName', 'StartDate', 'ExitDate', 'Title', 'Supervisor', 'ADEmail', 'BusinessUnit', 'EmployeeStatus', 'EmployeeType', 'PayZone', 'EmployeeClassificationType', 'TerminationType', 'TerminationDescription', 'DepartmentType', 'Division', 'DOB', 'State', 'JobFunctionDescription', 'GenderCode', 'LocationCode', 'RaceDesc', 'MaritalDesc', 'Performance Score', 'Current Employee Rating', 'Employee ID', 'Survey Date', 'Engagement Score', 'Satisfaction Score', 'Work-Life Balance Score', 'Training Date', 'Training Program Name', 'Training Type', 'Training Outcome', 'Location', 'Trainer', 'Training Duration(Days)', 'Training Cost']
Column: StartDate
Column: ExitDate
Column: DOB
Column: Survey Date
Column: Training Date
****************************** 

DF Information:  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3150 entries, 0 to 3149
Data columns (total 39 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                    

In [46]:
def turnNumeric(df,col):
    unique = {}
    for i,n in enumerate(df[col].unique()):
        unique[n] = i
    df[col] = df[col].apply(lambda x: unique[x])
    return df

In [52]:
df = turnNumeric(df,'Title')
df = turnNumeric(df,'EmployeeStatus')
df = turnNumeric(df,'Training Type')
df = turnNumeric(df,'Training Outcome')
df = turnNumeric(df,'Title')
df.describe()

,Unnamed: 0,StartDate,ExitDate,Title,EmployeeStatus,DOB,LocationCode,Current Employee Rating,Employee ID,Survey Date,Engagement Score,Satisfaction Score,Work-Life Balance Score,Training Date,Training Type,Training Outcome,Training Duration(Days),Training Cost
count,3150.000000,3150,1606,3150.000000,3150.000000,3150,3150.000000,3150.000000,3150.000000,3150,3150.000000,3150.000000,3150.000000,3150,3150.000000,3150.000000,3150.000000,3150.000000
mean,1495.748254,2021-01-29 16:56:41.142857216,2022-05-22 13:04:33.474470656,5.006349,0.409841,1971-08-17 17:52:54.857142856,45155.332698,2.968254,2502.748254,2023-02-04 12:02:17.142856960,2.943492,3.021270,2.992381,2023-02-03 02:40:54.857142784,0.496190,1.514921,2.971746,560.525889
min,0.000000,2018-08-07 00:00:00,2018-11-19 00:00:00,0.000000,0.000000,1941-02-10 00:00:00,1013.000000,1.000000,1001.000000,2022-01-09 00:00:00,1.000000,1.000000,1.000000,2022-08-05 00:00:00,0.000000,0.000000,1.000000,100.040000
25%,746.000000,2019-10-29 00:00:00,2021-10-11 06:00:00,0.000000,0.000000,1956-07-19 18:00:00,17616.500000,2.000000,1750.250000,2022-10-23 00:00:00,2.000000,2.000000,2.000000,2022-11-04 00:00:00,0.000000,1.000000,2.000000,331.475000
50%,1495.500000,2021-02-04 12:00:00,2022-09-16 00:00:00,1.000000,0.000000,1971-11-28 12:00:00,44455.500000,3.000000,2504.500000,2023-02-14 00:00:00,3.000000,3.000000,3.000000,2023-02-03 00:00:00,0.000000,2.000000,3.000000,574.300000
75%,2246.750000,2022-04-30 18:00:00,2023-04-10 00:00:00,4.000000,0.000000,1987-06-28 06:00:00,71713.250000,3.000000,3254.750000,2023-05-28 00:00:00,4.000000,4.000000,4.000000,2023-05-02 00:00:00,1.000000,2.000000,4.000000,788.070000
max,2999.000000,2023-08-06 00:00:00,2023-08-06 00:00:00,31.000000,4.000000,2001-11-04 00:00:00,98052.000000,5.000000,4000.000000,2023-12-07 00:00:00,5.000000,5.000000,5.000000,2023-08-05 00:00:00,1.000000,3.000000,5.000000,999.970000
std,867.007861,NaN,NaN,8.163926,0.943278,NaN,30019.418852,1.020717,866.241991,NaN,1.430727,1.407413,1.409244,NaN,0.500065,1.107267,1.415390,262.485994


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3150 entries, 0 to 3149
Data columns (total 39 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Unnamed: 0                  3150 non-null   int64         
 1   FirstName                   3150 non-null   object        
 2   LastName                    3150 non-null   object        
 3   StartDate                   3150 non-null   datetime64[ns]
 4   ExitDate                    1606 non-null   datetime64[ns]
 5   Title                       3150 non-null   int64         
 6   Supervisor                  3150 non-null   object        
 7   ADEmail                     3150 non-null   object        
 8   BusinessUnit                3150 non-null   object        
 9   EmployeeStatus              3150 non-null   int64         
 10  EmployeeType                3150 non-null   object        
 11  PayZone                     3150 non-null   object      

In [15]:
print(df.isna().sum())

Unnamed: 0                       0
FirstName                        0
LastName                         0
StartDate                        0
ExitDate                      1544
Title                            0
Supervisor                       0
ADEmail                          0
BusinessUnit                     0
EmployeeStatus                   0
EmployeeType                     0
PayZone                          0
EmployeeClassificationType       0
TerminationType                  0
TerminationDescription        1544
DepartmentType                   0
Division                         0
DOB                              0
State                            0
JobFunctionDescription           0
GenderCode                       0
LocationCode                     0
RaceDesc                         0
MaritalDesc                      0
Performance Score                0
Current Employee Rating          0
Employee ID                      0
Survey Date                      0
Engagement Score    